In [1]:
import pandas as pd
import pickle
from gop import *
from scipy.special import softmax
from kaldiio import ReadHelper

ModuleNotFoundError: No module named 'kaldiio'

In [ ]:
print(pd.__version__)

In [4]:
df_phones_pure = pd.read_pickle('phones_pure_epa.pickle')
df_phones_pure = df_phones_pure.reset_index()
df_phones_pure

,phone,phone_name,transition_state,hmm_state,pdf,transition_ids,phone_pure,phone_pure_name
0,15,AA0_B,263,0,2,525,3,AA
1,15,AA0_B,263,0,2,526,3,AA
2,15,AA0_B,264,0,225,527,3,AA
3,15,AA0_B,264,0,225,528,3,AA
4,15,AA0_B,265,0,225,529,3,AA
...,...,...,...,...,...,...,...,...
43985,342,Z_S,21935,0,5996,43870,40,Z
43986,342,Z_S,21936,0,5996,43871,40,Z
43987,342,Z_S,21936,0,5996,43872,40,Z
43988,342,Z_S,21937,0,5996,43873,40,Z


In [5]:
df_target = pd.read_pickle('target_epa.pickle')
df_target

array(['phones', 'phones_names', 'align_transitions', 'target',
       'target_base'], dtype=object)

In [6]:
matrix_gop_robust(df_phones_pure, 6024,1)

[[array([1., 0., 0., ..., 0., 0., 0.]),
  array([0., 1., 0., ..., 0., 0., 0.]),
  array([0., 0., 1., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),


In [7]:
alignments = pd.read_pickle('alignments.pickle')
alignments = pd.DataFrame.from_dict(alignments)
pd.DataFrame(alignments.loc[:,'spkr32_58']).transpose()

,phones,transitions
spkr32_58,"[SIL, DH_B, AH0_E, SIL, L_B, OW1_I, ER0_E, F_B...","[[2], [14148], [5944], [2, 1, 1, 1, 1, 1, 1, 1..."


In [8]:
gop = {}
with ReadHelper('ark:loglikes.ark') as reader:
    for key, loglikes in reader:
        
        loglikes = softmax(np.array(loglikes))
        df_scores = pd.DataFrame(alignments.loc[:,key]).transpose()
        df_scores['p'] = [loglikes]
        gop[key] = gop_robust_with_matrix(df_scores, df_phones_pure, 6024, 1, [], [])

with open('gop_epa.pickle', 'wb') as handle:
    pickle.dump(gop, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:
gop_epa = pd.read_pickle('gop_epa.pickle')


In [10]:
gop_output_file = open('gop_epa.txt', 'w+')

for logid, score in gop_epa.items():
    score = score[0]
    phones = score['phones_pure']
    gop = score['gop']
    
    if len(phones) != len(gop):
        raise Exception("Phones and gop list lengths do not match.")
    
    gop_output_file.write(logid)
    for i in range(len(phones)):
        gop_output_file.write(' [ ' + phones[i] + ' ' + str(gop[i]) + ' ] ')
    gop_output_file.write('\n')
